In [47]:
import pandas as pd
import numpy as np

# Definitions
* **Bus line**:

    Defined as: (bus_service_number,direction) 
    
    There are 2 directions for each bus service, 1 and 2.

* **Distance between MRT from a Bus Stop**:

    Defined using Euclidean Distance

    This a proxy for walking distance.

# Approach
We will split the work into multiple notebooks for faster computation and debugging

* get_nearest_mrt_to_bus_stops.ipynb -> ETL to get bus_stops_with_nearest_mrt_data.csv

    The new csv will have at least the following columns: 
     - Bus Stop Code
     - MRT Station Name
     - CCL_station_name
     - CCL_distance
     - NSL_station_name
     - NSL_distance
     ... (we will have 2 columns for each MRT line)
    
    Primary Key is Bus Stop Code.

* Bus Algorithm.ipynb -> dictionary of scores



In [21]:
bus_routes_df = pd.read_csv('data/bus_routes.csv')
bus_routes_df.head()

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309


In [22]:
bus_stops_with_nearest_mrt_df = pd.read_csv('processed_data/bus_stops_with_nearest_mrt_data.csv')
bus_stops_with_nearest_mrt_df.head()

,BusStopCode,RoadName,Description,Latitude,Longitude,BPLRT_distance,BPLRT_station_name,CCL_distance,CCL_station_name,DTL_distance,...,NEL_distance,NEL_station_name,NSL_distance,NSL_station_name,PGLRT_distance,PGLRT_station_name,SGLRT_distance,SGLRT_station_name,TEL_distance,TEL_station_name
0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536,12627.645635,Pending,593.415220,Bras Basah,360.474492,...,830.710778,Dhoby Ghaut,371.033933,Somerset,1871.146185,Nibong,10381.520180,Fernvale,1917.338530,Gardens By The Bay
1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225,12616.136712,Pending,471.411686,Bras Basah,239.135214,...,877.652738,Dhoby Ghaut,368.800944,Somerset,1903.571130,Nibong,10272.490723,Fernvale,1935.477848,Gardens By The Bay
2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022,12655.453666,Pending,545.294931,Bras Basah,320.881807,...,876.763197,Dhoby Ghaut,397.758557,Somerset,1851.135676,Nibong,10355.453514,Fernvale,1891.161808,Gardens By The Bay
3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414,12791.429559,Pending,490.588813,Bras Basah,326.541656,...,1035.017656,Dhoby Ghaut,459.503618,City Hall,1734.383323,Nibong,10361.025686,Fernvale,1761.810539,Gardens By The Bay
4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491,12761.714232,Pending,288.150419,Bras Basah,208.321318,...,1117.203665,Dhoby Ghaut,585.584287,Somerset,1823.333386,Nibong,10171.916411,Fernvale,1825.804060,Gardens By The Bay


In [23]:
# Convert df to dict for fast look up
bus_stops_with_nearest_mrt_df.set_index('BusStopCode', inplace=True)
bus_stops_with_nearest_mrt_dict = bus_stops_with_nearest_mrt_df.to_dict(orient='index')
# E.g.
bus_stops_with_nearest_mrt_dict[17091]

{'RoadName': 'Clementi Rd',
 'Description': 'Aft Clementi Ave 1',
 'Latitude': 1.30904805161426,
 'Longitude': 103.77157783850733,
 'BPLRT_distance': 7462.235844557487,
 'BPLRT_station_name': 'Pending',
 'CCL_distance': 2042.3107185462184,
 'CCL_station_name': 'One-North',
 'DTL_distance': 2711.8760874703285,
 'DTL_station_name': 'King Albert Park',
 'EWL_distance': 1345.036173889697,
 'EWL_station_name': 'Dover',
 'NEL_distance': 7413.860801613406,
 'NEL_station_name': 'Harbourfront',
 'NSL_distance': 4249.411619989203,
 'NSL_station_name': 'Jurong East',
 'PGLRT_distance': 3159.473265091157,
 'PGLRT_station_name': 'Oasis',
 'SGLRT_distance': 14155.159113654605,
 'SGLRT_station_name': 'Fernvale',
 'TEL_distance': 5274.605123069488,
 'TEL_station_name': 'Napier'}

In [48]:
busline_scores = {} #Key: (bus_service_number,direction), Value: hashmap of MRT line scores.
#Hashmap of MRT line scores will have: key: MRT line, value: score

for _, row in bus_routes_df[["ServiceNo","Direction"]].drop_duplicates().iterrows(): # 1 iteration per bus line
    service_number = row['ServiceNo']
    direction = row['Direction']
    mrt_line_scores = {}
    for mrt_line in ['BPLRT', 'CCL', 'DTL', 'EWL', 'NEL', 'NSL', 'PGLRT', 'SGLRT','TEL']:
        distances_to_mrt = []
        bus_stops = bus_routes_df.loc[(bus_routes_df["ServiceNo"] == service_number)
                                      & (bus_routes_df["Direction"] == direction),
                                      "BusStopCode"]
        for bus_stop in bus_stops:
            # Get the distance to the nearest MRT station in the MRT line
            distance = bus_stops_with_nearest_mrt_dict[bus_stop][f"{mrt_line}_distance"]
            distances_to_mrt.append(distance)

        mrt_line_scores[mrt_line] = int(np.median(distances_to_mrt)) # round
    #Sort mrt_line_scores by value
    mrt_line_scores = {k: v for k, v in sorted(mrt_line_scores.items(), key=lambda item: item[1])}
    busline_scores[(service_number,direction)] = mrt_line_scores


In [52]:
# We will sort the bus lines base on the minimum score of the MRT lines
busline_scores = dict(sorted(busline_scores.items(), key=lambda kv_pair: min(kv_pair[1].values())))

# Print first 50 bus lines
for i, (busline, mrt_scores) in enumerate(busline_scores.items()):
    if i == 50:
        break
    print(busline, mrt_scores)

('384', 1) {'PGLRT': 190, 'NEL': 712, 'SGLRT': 1601, 'CCL': 6732, 'NSL': 7143, 'DTL': 7176, 'EWL': 7785, 'TEL': 7907, 'BPLRT': 14368}
('163A', 1) {'SGLRT': 193, 'NEL': 902, 'PGLRT': 2050, 'CCL': 4212, 'NSL': 5473, 'DTL': 5933, 'TEL': 6179, 'EWL': 6487, 'BPLRT': 13257}
('374', 1) {'SGLRT': 210, 'NEL': 1063, 'PGLRT': 1484, 'CCL': 4787, 'NSL': 5546, 'TEL': 6263, 'DTL': 6689, 'EWL': 7368, 'BPLRT': 12866}
('991B', 1) {'BPLRT': 217, 'NSL': 680, 'DTL': 1805, 'EWL': 4460, 'TEL': 5623, 'CCL': 9318, 'PGLRT': 11443, 'SGLRT': 13835, 'NEL': 13975}
('973A', 1) {'BPLRT': 220, 'DTL': 807, 'NSL': 1795, 'EWL': 5391, 'TEL': 5826, 'CCL': 8101, 'PGLRT': 10464, 'SGLRT': 11553, 'NEL': 11834}
('114A', 1) {'NEL': 248, 'SGLRT': 800, 'PGLRT': 2980, 'CCL': 3248, 'NSL': 4458, 'TEL': 4940, 'DTL': 5433, 'EWL': 5938, 'BPLRT': 12105}
('976', 2) {'BPLRT': 262, 'DTL': 1231, 'NSL': 1705, 'EWL': 5227, 'TEL': 5520, 'CCL': 8782, 'PGLRT': 11129, 'SGLRT': 12052, 'NEL': 12428}
('920', 1) {'BPLRT': 268, 'DTL': 1020, 'NSL': 1988